In [68]:
import requests, re
from bs4 import BeautifulSoup
from IPython.display import HTML, display

import pandas as pd

# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss
# goes to page 278

# first page of thread:

# store data in lists --> pandas dataframe

# THREAD NAME
# <h3 class="post-title"><a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941">I can&#039;t live without my dad - anyone feel the same?</a></h3>

# THREAD ADDRESS
# <link rel="canonical"
# href="https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same" />

    # NEW:
    
    # <li class="cr-pagination__item--current">
    # <a title="Go to page 5"
    # href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=4">5</a></li>

# USERNAME
# <a href="/about-cancer/cancer-chat/users/bras1548" title="View user profile." class="username">bras1548</a>

# POST DATE
# <span class="post-created hidden-xs">22 Jun 2020 16:35</span>

# POST TEXT
# <div class="field field-name-field-harmony-text field-type-text-long field-label-hidden">

# POST IS A REPLY TO POST_ID
# <span class="post-is-reply-to">22 Jun 2020 21:04 in response to <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941" class="active">bras1548</a></span>

# POST ID
# <a name="post-1305941"></a>



In [111]:
# FUNCTION: SCRAPE A THREAD PAGE
def page_scraper_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')
    
    # THREAD ADDRESS
    
    # <li class="cr-pagination__item--current">
    # <a title="Go to page 5"
    # href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=4">5</a></li>
    
    thread_address_list = []
    thread_address = soup.find("link",{"rel":"canonical"})['href']
    #print(thread_address)

    # THREAD NAME
    thread_name_list = []
    thread_group = soup.find("h3", {"class": "post-title"})
    thread_name = thread_group.find("a").text
    #print(thread_name)

    # POST ID
    post_id_list = []
    post_id_group = soup.findAll("a", {"name": re.compile(r".*")})

    for post_id_g in post_id_group:
        if "post-" in post_id_g["name"]:
            post_id_list.append(post_id_g["name"])
            #print(post_id_g["name"])

    # POST USERNAME
    username_list = []
    user_group = soup.findAll("a", {"class": "username"})

    for user_g in user_group:    
        username = user_g.text
        username_list.append(username)

        # populate THREAD ADDRESS, THREAD NAME lists
        thread_address_list.append(thread_address)
        thread_name_list.append(thread_name)
        #print(username)

    # POST DATE
    date_list = []
    date_group = soup.findAll("span", {"class": "post-created hidden-xs"})

    for date_g in date_group:
        date = date_g.text
        date_list.append(date)
        #print(date)

    # POST TEXT
    text_list = []
    text_group = soup.findAll("div", {"class": "field field-name-field-harmony-text field-type-text-long field-label-hidden"})

    for text_g in text_group:
        text_tag = text_g.find("div").find("div")
        text = text_tag.text
        text_list.append(text)
        #print(text_tag.text, "\n")

    # POST IS A REPLY TO
    reply_to_list = []
    reply_group = soup.findAll("span", {"class": "post-is-reply-to"})

    # add "N/A only to first post in thread
    if len(reply_group) < len(text_group):
        reply_to_list.append("N/A")
    
    for reply_g in reply_group:
        reply_to_id = reply_g.find("a")['href']
        reply_to_id = "https://www.cancerresearchuk.org" + reply_to_id
        reply_to_id = reply_to_id.replace(thread_address + "#", "")
        reply_to_list.append(reply_to_id)
        #print(reply_to_id)

    # COLUMN LISTS INTO A LIST OF ROWS
    new_rows = []

    for j in range(len(thread_name_list)):
        row_list = [thread_name_list[j], thread_address_list[j], username_list[j],
                    post_id_list[j], date_list[j], text_list[j], reply_to_list[j]]

        new_rows.append(row_list)
    
    return new_rows




In [99]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

# <li class="cr-pagination__next">
# <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1">Next</a></li>

def next_page_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')

    next_link = soup.find("li", {"class":"cr-pagination__next"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org/" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=4#post-1515206"))
print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"))



None
https://www.cancerresearchuk.org//about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1


In [86]:
# TEST PAGE SCRAPER FUNCTION

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"

new_rows = page_scraper_cancer_research(page_link)


In [108]:
for row in new_rows:
    
    print(row[0])
    print(row[1])
    print(row[2])
    print(row[3])
    print(row[4])
    print(row[5])
    print(row[6], "\n")

    

I can't live without my dad - anyone feel the same?
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same
bras1548
post-1305941
22 Jun 2020 16:35
Hi everyone, My father died almost 6 months ago. He was my best friend, my soulmate, my inspiration, my confidant, basically the love of my life. I will never love anyone that way again. He was my other half. We communicated almost every single day in some way (although I have regrets about how I handled this, which I may share another time). The point, though, is this: I don't think I can live without him. He meant more to me than life itself. He was my oxygen, my sunshine, my everything. He was the most remarkable and wonderful human being I have ever known (and will ever know). Every day now I have the pain of realising that he won't ever be with me again. I'm writing this post for one reason: to find other people who feel the same or at least similar. Here are some things that I h

In [112]:
# FUNCTION: SCRAPE ONE THREAD

row_list = []
page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same"

def thread_scraper_cancer_research(page_link):

    new_rows = page_scraper_cancer_research(page_link)
    next_link = next_page_cancer_research(page_link)
    
    if next_link is not None:
        new_rows.extend(thread_scraper_cancer_research(next_link))

    return new_rows
    


In [113]:
new_rows = thread_scraper_cancer_research(page_link)
print(len(new_rows))

63


In [114]:
# GLOBAL database for 
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_coping_w_loss = pd.DataFrame(new_rows, columns = column_names)  
print (cancer_research_coping_w_loss)

cancer_research_coping_w_loss.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_test.csv', sep ='\t', index = False, header=True)


                                          Thread_name  \
0   I can't live without my dad - anyone feel the ...   
1   I can't live without my dad - anyone feel the ...   
2   I can't live without my dad - anyone feel the ...   
3   I can't live without my dad - anyone feel the ...   
4   I can't live without my dad - anyone feel the ...   
5   I can't live without my dad - anyone feel the ...   
6   I can't live without my dad - anyone feel the ...   
7   I can't live without my dad - anyone feel the ...   
8   I can't live without my dad - anyone feel the ...   
9   I can't live without my dad - anyone feel the ...   
10  I can't live without my dad - anyone feel the ...   
11  I can't live without my dad - anyone feel the ...   
12  I can't live without my dad - anyone feel the ...   
13  I can't live without my dad - anyone feel the ...   
14  I can't live without my dad - anyone feel the ...   
15  I can't live without my dad - anyone feel the ...   
16  I can't live without my dad